In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from IPython.display import display

#Chrome Options
options = Options()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--start-maximized')
options.add_argument('--allow-running-insecure-content')
options.add_experimental_option('excludeSwitches', ['enable-automation'])

#chrome Setting headless
# options = Options()
# options.add_argument('--headless')  # ←   headless windows
# options.add_argument('--disable-gpu')  # ← to prevent bugs
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--start-maximized')
# options.add_argument('--allow-running-insecure-content')
# options.add_experimental_option('excludeSwitches', ['enable-automation'])


driver = webdriver.Chrome(options=options)
driver.get("https://www.time.ir/")
wait = WebDriverWait(driver, 15)

time.sleep(2)

# Handle SSL error if shown
if "Your connection is not private" in driver.page_source:
    driver.execute_script("document.querySelector('button#details-button').click()")
    time.sleep(1)
    driver.execute_script("document.querySelector('#proceed-link').click()")
    time.sleep(2)

# Click on year dropdown
butt1_path = '#calendar > div.CalendarBox_root__E_XXm.EventCalendar_root__calendarDays__uH4Zk > div.CalendarDetails_root__NwPpd.false > div.CalendarDetails_root__date__Ova57 > div:nth-child(1) > div:nth-child(1)'
button1 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, butt1_path)))
button1.click()
time.sleep(1)

# Click on Farvardin (first month)
butt2_path = '#calendar > div.CalendarBox_root__E_XXm.EventCalendar_root__calendarDays__uH4Zk > div.CalendarDetails_root__NwPpd.false > div.CalendarDetails_root__date__Ova57 > div:nth-child(1) > div.MuiCollapse-root.CalendarPickerCollapse_root__XqHYd.MuiCollapse-vertical.MuiCollapse-entered.muirtl-1cbf1l2 > div > div > div > div:nth-child(1) > div'
button2 = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, butt2_path)))
driver.execute_script("arguments[0].scrollIntoView({block: 'nearest'});", button2)
time.sleep(0.5)
button2.click()

time.sleep(2)

# Number of months to extract
num_months = 12

# Lists to collect data
all_date = []
all_data_gham = []
all_dates_fa = []
all_events_fa = []
all_events_en = []
all_holiday = []

for i in range(num_months):
    print(f"📅 Collecting events for month {i+1}...")
    time.sleep(1)

    # Read current year/month
    try:
        ym_elem = driver.find_element(By.CLASS_NAME, 'CalendarDetailItem_root__jalali__JpHKf')
        year_month = ym_elem.text.strip()
    except:
        year_month = " "

    
    try:
        ym_el_gham = driver.find_element(By.CSS_SELECTOR,"#calendar > div.CalendarBox_root__E_XXm.EventCalendar_root__calendarDays__uH4Zk > div.CalendarDetails_root__NwPpd.false > div.CalendarDetails_root__date__Ova57 > div:nth-child(3)")
        year_month_gham = ym_el_gham.text.strip()
    except:
        year_month_gham = " "
    # Get all events for the month
    items = driver.find_elements(By.CLASS_NAME, "EventListItem_root__pHV2b")

    for item in items:
        try:
            date_fa = item.find_element(By.CLASS_NAME, "EventListItem_root__date__UUgtf").text.strip()
        except:
            date_fa = " "

        try:
            event_elem = item.find_element(By.CLASS_NAME, "EventListItem_root__event__XrjoV ")
            event_fa = event_elem.text.strip()

        #************* ---------------------------------------------------------------------------------------------------------------------------****
            event_classes = event_elem.get_attribute("class")
            is_holiday = 1 if "EventListItem_root__event__holiday__vbVe2" in event_classes else 0

        except:
            event_fa = " "
            
        try:
            event_en = item.find_element(By.CLASS_NAME, "EventListItem_root__otherBase__8Sksv").text.strip()
        except:
            event_en = " "

        all_date.append(year_month)
        all_data_gham.append(year_month_gham)
        all_dates_fa.append(date_fa)
        all_events_fa.append(event_fa)
        all_events_en.append(event_en)
        all_holiday.append(is_holiday)

    # Move to next month
    try:
        next_button = wait.until(EC.element_to_be_clickable((
            By.CSS_SELECTOR,
            "#calendar > div.CalendarBox_root__E_XXm.EventCalendar_root__calendarDays__uH4Zk > div.CalendarDetails_root__NwPpd.false > div.CalendarDetails_root__next__xtjZ_ > svg"
        )))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
        time.sleep(0.5)
        next_button.click()
    except Exception as e:
        print(f"❗ Error clicking next month at iteration {i+1}: {e}")
        break

# Close the browser
driver.quit()

# Create DataFrame
df = pd.DataFrame({
    'Year and Month': all_date,
    'Year month qhamari': all_data_gham,
    'Persian Date': all_dates_fa,
    'Event Title': all_events_fa,
    'Gregorian/Islamic Date': all_events_en,
    'Holiday' : all_holiday
})

# Optional: display clean styled table in Jupyter
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

styled_df = df.style.set_properties(**{
    'text-align': 'right',
    'direction': 'rtl',
    'font-family': 'Tahoma',
    'border': '1px solid #ccc',
    'font-size': '13px',
    'padding': '2px'
}).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]}
])

display(styled_df)


📅 Collecting events for month 1...
📅 Collecting events for month 2...
📅 Collecting events for month 3...
📅 Collecting events for month 4...
📅 Collecting events for month 5...
📅 Collecting events for month 6...
📅 Collecting events for month 7...
📅 Collecting events for month 8...
📅 Collecting events for month 9...
📅 Collecting events for month 10...
📅 Collecting events for month 11...
📅 Collecting events for month 12...


,Year and Month,Year month qhamari,Persian Date,Event Title,Gregorian/Islamic Date,Holiday
0,فروردین 1404,رمضان - شوال ١٤٤٦,1 فروردین,جشن نوروز/جشن سال نو,,1
1,فروردین 1404,رمضان - شوال ١٤٤٦,1 فروردین,روز جهانی نوروز,[ March 21 ],0
2,فروردین 1404,رمضان - شوال ١٤٤٦,2 فروردین,عیدنوروز,,1
3,فروردین 1404,رمضان - شوال ١٤٤٦,2 فروردین,شهادت حضرت علی (ع),[ ۲۱ رمضان ],1
4,فروردین 1404,رمضان - شوال ١٤٤٦,3 فروردین,عیدنوروز,,1
5,فروردین 1404,رمضان - شوال ١٤٤٦,3 فروردین,شب قدر,[ ۲۲ رمضان ],0
6,فروردین 1404,رمضان - شوال ١٤٤٦,3 فروردین,روز جهانی هواشناسی,[ March 23 ],0
7,فروردین 1404,رمضان - شوال ١٤٤٦,4 فروردین,عیدنوروز,,1
8,فروردین 1404,رمضان - شوال ١٤٤٦,6 فروردین,روز امید، روز شادباش نویسی,,0
9,فروردین 1404,رمضان - شوال ١٤٤٦,6 فروردین,زادروز اَشو زرتشت، اَبَراِنسان بزرگ تاریخ,,0


In [9]:
print(f"all_date: {len(all_date)}")
print(f"all_data_gham: {len(all_data_gham)}")
print(f"all_dates_fa: {len(all_dates_fa)}")
print(f"all_events_fa: {len(all_events_fa)}")
print(f"all_events_en: {len(all_events_en)}")
print(f"all_holiday: {len(all_holiday)}")
print(all_holiday)

all_date: 234
all_data_gham: 234
all_dates_fa: 234
all_events_fa: 234
all_events_en: 234
all_holiday: 234
[1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]


In [10]:
from convertdate import persian, islamic

shamsi_year = list(map(lambda x:x.split('\n')[1],all_date))
shamsi_alpha_month = list(map(lambda x:x.split('\n')[0],all_date))
months_dict = {
    'فروردین': 1,
    'اردیبهشت': 2,
    'خرداد': 3,
    'تیر': 4,
    'اَمرداد': 5,  # ← اصلاح شد (کلید باید 'اَمرداد' باشه)
    'شهریور': 6,
    'مهر': 7,
    'آبان': 8,
    'آذر': 9,
    'دی': 10,
    'بهمن': 11,
    'اسفند': 12
}

#Build A dataframe for Convert Dates
shamsi_data = pd.DataFrame([shamsi_year,shamsi_alpha_month]).T
shamsi_data.columns = ['shamsi_year','shamsi_alpha_month']
shamsi_data['shamsi_month'] = pd.to_numeric(
    shamsi_data['shamsi_alpha_month'].map(months_dict),
    errors='coerce'
).astype('Int64')

#DAY
shamsi_day = list(map(lambda x:x.split(' ')[0],all_dates_fa))
shamsi_data['shamsi_day'] = shamsi_day

#Function To convert Dates in Jalali to Hejra and gregorian
def convert_to_ghamari(y, m, d):

    # convertdate.iranian is for Jalali (Shamsi)
    g_year, g_month, g_day = persian.to_gregorian(y, m, d)

    # Now convert to Hijri
    h_year, h_month, h_day = islamic.from_gregorian(g_year, g_month, g_day)

    return [g_year, g_month, g_day, h_year, h_month, h_day]



shamsi_data['miladi_ghamari'] = shamsi_data.apply(lambda x:convert_to_ghamari(int(x['shamsi_year']),int(x['shamsi_month']),int(x['shamsi_day'])),axis=1)
shamsi_data['miladi'] = shamsi_data['miladi_ghamari'].apply(lambda x:str(x[0])+'/'+str(x[1])+'/'+str(x[2]))
shamsi_data['ghamari'] = shamsi_data['miladi_ghamari'].apply(lambda x:str(x[3])+'/'+str(x[4])+'/'+str(x[5]))

#shamsi
shamsi_data['shamsi'] = (
    shamsi_data['shamsi_year'].astype(str) + '/' +
    shamsi_data['shamsi_month'].astype(str).str.zfill(2) + '/' +
    shamsi_data['shamsi_day'].astype(str).str.zfill(2)
)
#shamsi_data['shamsi']

#shamsi_data
df_merged = pd.concat([df, shamsi_data[['miladi', 'ghamari','shamsi']].reset_index(drop=True)], axis=1)
df_merged.drop(columns=['Year and Month','Year month qhamari','Persian Date'],inplace = True)
df_merged

,Event Title,Gregorian/Islamic Date,Holiday,miladi,ghamari,shamsi
0,جشن نوروز/جشن سال نو,,1,2025/3/21,1446/9/21,1404/01/01
1,روز جهانی نوروز,[ March 21 ],0,2025/3/21,1446/9/21,1404/01/01
2,عیدنوروز,,1,2025/3/22,1446/9/22,1404/01/02
3,شهادت حضرت علی (ع),[ ۲۱ رمضان ],1,2025/3/22,1446/9/22,1404/01/02
4,عیدنوروز,,1,2025/3/23,1446/9/23,1404/01/03
5,شب قدر,[ ۲۲ رمضان ],0,2025/3/23,1446/9/23,1404/01/03
6,روز جهانی هواشناسی,[ March 23 ],0,2025/3/23,1446/9/23,1404/01/03
7,عیدنوروز,,1,2025/3/24,1446/9/24,1404/01/04
8,روز امید، روز شادباش نویسی,,0,2025/3/26,1446/9/26,1404/01/06
9,زادروز اَشو زرتشت، اَبَراِنسان بزرگ تاریخ,,0,2025/3/26,1446/9/26,1404/01/06


In [11]:
gregorian_months = [
    'January', 'February', 'March', 'April',
    'May', 'June', 'July', 'August',
    'September', 'October', 'November', 'December'
]
hijri_months = [
    'محرم',         # Muharram
    'صفر',           # Safar
    'ربیع‌الاول',    # Rabi' al-awwal
    'ربیع‌الثانی',   # Rabi' al-thani
    'جمادی‌الاول',   # Jumada al-awwal
    'جمادی‌الثانی',  # Jumada al-thani
    'رجب',          # Rajab
    'شعبان',        # Sha'ban
    'رمضان',        # Ramadan
    'شوال',         # Shawwal
    'ذی‌القعده',     # Dhu al-Qa'dah
    'ذی‌الحجه'       # Dhu al-Hijjah
]


In [12]:
def flag(text):
    if any(month in text for month in hijri_months):
        return 1
    elif any(month in text for month in gregorian_months):
        return 2
    else:
        return 0
        

In [16]:
df_merged['flag'] = df_merged['Gregorian/Islamic Date'].apply(flag)
df_merged[['shamsi','ghamari','miladi','Event Title','Holiday','flag']].to_excel('holiday_data.xlsx')
